In [6]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
from tqdm import tqdm
sys.path.insert(0,'/Data2/hmd/hmd_sy/evaluation-2022')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *

2022-07-20 01:59:28.023120: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
root_dir = 'physionet.org/files/circor-heart-sound/1.0.3'
training_data_file = root_dir + '/' + 'training_data.csv'
training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [7]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Wed Jul 20 06:45:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.39       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:1A:00.0 Off |                  Off |
| 39%   63C    P2   153W / 260W |  48148MiB / 48601MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Quadro RTX 8000     On   | 00000000:3D:00.0 Off |                  Off |
| 49%   

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
    except RuntimeError as e:
        print(e)

2022-07-18 23:49:53.572992: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-07-18 23:49:53.628426: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:1a:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.46GiB deviceMemoryBandwidth: 625.94GiB/s
2022-07-18 23:49:53.630183: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:3d:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.46GiB deviceMemoryBandwidth: 625.94GiB/s
2022-07-18 23:49:53.631903: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 2 with properties: 
pciBusID: 0000:3e:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.46GiB deviceMemoryBandwidth: 625.94GiB/s
2022-07-18 23:49

In [6]:
data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/Data2/hmd/data_split/murmur/train'
test_folder = '/Data2/hmd/data_split/murmur/test'


In [7]:
model_folder = 'tmp_model0'

# Feature 추출



In [8]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

In [9]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

In [10]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [11]:
params_feature = {'samp_sec': 20,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': 256,
                  'win_length':512,
                  'n_mels': 100,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10
}

In [12]:
features_trn, mm_lbs_trn, out_lbs_trn, mel_input_shape, cqt_input_shape, stft_input_shape = get_features_3lb_all(train_folder, patient_files_trn, **params_feature)
features_test, mm_lbs_test, out_lbs_test, _, _, _ = get_features_3lb_all(test_folder, patient_files_test, **params_feature)

100%|██████████| 751/751 [04:06<00:00,  3.04it/s]


melspec:  100 313
cqt:  80 157
stft:  257 313


/home/ubuntu/anaconda3/envs/tensorflow2.4_p37/lib/python3.7/site-packages/ipykernel/__main__.py:150: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
100%|██████████| 191/191 [01:02<00:00,  3.04it/s]


melspec:  100 313
cqt:  80 157
stft:  257 313


## 추출한 피쳐 피클로 저장

In [18]:
with open('features_trn.pkl','wb') as f:
    pickle.dump(features_trn,f)
    
with open('mm_lbs_trn.pkl','wb') as f:
    pickle.dump(mm_lbs_trn,f)
    
with open('out_lbs_trn.pkl','wb') as f:
    pickle.dump(out_lbs_trn,f)
    
with open('mel_input_shape.pkl','wb') as f:
    pickle.dump(mel_input_shape,f)

with open('cqt_input_shape.pkl','wb') as f:
    pickle.dump(cqt_input_shape,f)
    
with open('stft_input_shape.pkl','wb') as f:
    pickle.dump(stft_input_shape,f)
    
with open('features_test.pkl','wb') as f:
    pickle.dump(features_test,f)
    
with open('mm_lbs_test.pkl','wb') as f:
    pickle.dump(mm_lbs_test,f)

with open('out_lbs_test.pkl','wb') as f:
    pickle.dump(out_lbs_test,f)


## 재시작할 때 불러오기

In [13]:
import pickle
with open('/Data2/hmd/hmd_sy/notebooks/features_trn.pkl','rb') as f:
    features_trn = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/mm_lbs_trn.pkl','rb') as f:
    mm_lbs_trn = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/out_lbs_trn.pkl','rb') as f:
    out_lbs_trn = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/mel_input_shape.pkl','rb') as f:
    mel_input_shape = pickle.load(f)

with open('/Data2/hmd/hmd_sy/notebooks/cqt_input_shape.pkl','rb') as f:
    cqt_input_shape = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/stft_input_shape.pkl','rb') as f:
    stft_input_shape = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/features_test.pkl','rb') as f:
    features_test = pickle.load(f)
    
with open('/Data2/hmd/hmd_sy/notebooks/mm_lbs_test.pkl','rb') as f:
    mm_lbs_test = pickle.load(f)

with open('/Data2/hmd/hmd_sy/notebooks/out_lbs_test.pkl','rb') as f:
    out_lbs_test = pickle.load(f)


In [11]:

print(mel_input_shape,cqt_input_shape,stft_input_shape)

(100, 313, 1) (80, 157, 1) (257, 313, 1)


# Modeling

* get_toy0_1, get_toy0_2: (3,3) -> (1,1)으로 감소    
* get_toy1_1, get_toy1_2: (3,3)
--------------------------
* feature concat code 수정 
    * 단일 feature인 경우, concat 함수 사용하면 에러 발생

In [14]:
def get_toy0_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
    cqt1 = keras.Input(shape=(cqt_input_shape), name = 'cqt')
    stft1 = keras.Input(shape=(stft_input_shape), name = 'stft')
        
        
    ## age embeddig
    age1 = layers.Dense(2, activation = None)(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = None)(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = None)(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = None)(loc)

    ## mel embedding
    mel2 = layers.Conv2D(16, (1,1), activation = 'relu')(mel1)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (5,5), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(64, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.GlobalAveragePooling2D()(mel2)

    ## cqt embedding
    cqt2 = layers.Conv2D(16, (1,1), activation = 'relu')(cqt1)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(32, (3,3), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(32, (5,5), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(64, (3,3), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.GlobalAveragePooling2D()(cqt2)

    ## stft embedding
    stft2 = layers.Conv2D(16, (1,1), activation = 'relu')(stft1)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(32, (3,3), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(32, (5,5), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(64, (3,3), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.GlobalAveragePooling2D()(stft2)
    
#    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, preg])
#    d1 = layers.Dense(2, activation = 'relu')(concat1)
    
    if use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2, stft2])
    if not use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([cqt2, stft2])
    if use_mel and not use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, stft2])
    if use_mel and use_cqt and not use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2])
        
    if not use_mel and not use_cqt and use_stft :  ## stft 만
        concat2 = stft2
    if use_mel and not use_cqt and not use_stft :  ### mel만
        concat2 = mel2
    if not use_mel and use_cqt and not use_stft :  ### cqt만
        concat2 = cqt2
#    concat2 = layers.Dense(10, activation = 'relu')(concat2)
    res1 = layers.Dense(3, activation = "softmax")(concat2)
    res2 = layers.Dense(2, activation = "softmax")(concat2)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1,cqt1,stft1] , outputs = res1 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
    return(model)

def get_toy0_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True):
        # Create a towy model.
    age = keras.Input(shape=(6,), name = 'age_cat')
    sex = keras.Input(shape=(2,), name = 'sex_cat')
    hw = keras.Input(shape=(2,), name = 'height_weight')
    preg = keras.Input(shape=(1,), name = 'is_preg')
    loc = keras.Input(shape=(5,), name = 'loc')
    mel1 = keras.Input(shape=(mel_input_shape), name = 'mel')
    cqt1 = keras.Input(shape=(cqt_input_shape), name = 'cqt')
    stft1 = keras.Input(shape=(stft_input_shape), name = 'stft')
        
        
    ## age embeddig
    age1 = layers.Dense(2, activation = None)(age)

    ## sex embedding
    sex1 = layers.Dense(1, activation = None)(sex)

    ## hw embedding
    hw1 = layers.Dense(1, activation = None)(hw)

    ## loc embedding
    loc1 = layers.Dense(3, activation = None)(loc)

    ## mel embedding
    mel2 = layers.Conv2D(16, (1,1), activation = 'relu')(mel1)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(32, (5,5), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.Conv2D(64, (3,3), activation = 'relu')(mel2)
    mel2 = layers.MaxPooling2D()(mel2)
    mel2 = layers.GlobalAveragePooling2D()(mel2)

    ## cqt embedding
    cqt2 = layers.Conv2D(16, (1,1), activation = 'relu')(cqt1)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(32, (3,3), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(32, (5,5), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.Conv2D(64, (3,3), activation = 'relu')(cqt2)
    cqt2 = layers.MaxPooling2D()(cqt2)
    cqt2 = layers.GlobalAveragePooling2D()(cqt2)

    ## stft embedding
    stft2 = layers.Conv2D(16, (1,1), activation = 'relu')(stft1)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(32, (3,3), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(32, (5,5), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.Conv2D(64, (3,3), activation = 'relu')(stft2)
    stft2 = layers.MaxPooling2D()(stft2)
    stft2 = layers.GlobalAveragePooling2D()(stft2)
    
#    concat1 = layers.Concatenate()([age1, sex1, hw1, loc1, preg])
#    d1 = layers.Dense(2, activation = 'relu')(concat1)
    
    if use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2, stft2])
    if not use_mel and use_cqt and use_stft :
        concat2 = layers.Concatenate()([cqt2, stft2])
    if use_mel and not use_cqt and use_stft :
        concat2 = layers.Concatenate()([mel2, stft2])
    if use_mel and use_cqt and not use_stft :
        concat2 = layers.Concatenate()([mel2, cqt2])
    if not use_mel and not use_cqt and use_stft :  ## stft만
        concat2 = stft2
    if use_mel and not use_cqt and not use_stft :  ### mel만
        concat2 = mel2
    if not use_mel and use_cqt and not use_stft :  ### cqt만
        concat2 = cqt2
#    concat2 = layers.Dense(10, activation = 'relu')(concat2)
    res1 = layers.Dense(3, activation = "softmax")(concat2)
    res2 = layers.Dense(2, activation = "softmax")(concat2)

    model = keras.Model(inputs = [age,sex,hw,preg,loc,mel1,cqt1,stft1] , outputs = res2 )
    
    model.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'])
    return(model)

* get_toy0_1, get_toy0_2: (3,3) -> (1,1)으로 감소    


# 실험 결과

## mel, cqt, stft 

In [ ]:
model_folder = 'tmp_model0_1'

In [15]:
model1 = get_toy0_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True)
model2 = get_toy0_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = True)


2022-07-18 05:50:49.942572: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-18 05:50:49.954723: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:3d:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 47.46GiB deviceMemoryBandwidth: 625.94GiB/s
2022-07-18 05:50:49.960822: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 1
2022-07-18 05:50:49.961196: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-07-18 05:50:50.825181: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect Str

In [53]:
# model1 = get_toy0_1(mel_input_shape, cqt_input_shape, stft_input_shape)
# model2 = get_toy0_2(mel_input_shape, cqt_input_shape, stft_input_shape)


In [16]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        mm_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             mm_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
                             epochs = n_epoch)

2022-07-18 05:51:13.626625: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-18 05:51:13.644855: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2100000000 Hz


Epoch 1/100


2022-07-18 05:51:14.491301: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-07-18 05:51:15.089329: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8100
2022-07-18 05:51:16.027721: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-07-18 05:51:16.548713: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


40/40 [==============================] - 12s 196ms/step - loss: 0.7625 - accuracy: 0.7535 - val_loss: 0.6068 - val_accuracy: 0.7496
Epoch 2/100
40/40 [==============================] - 6s 141ms/step - loss: 0.5739 - accuracy: 0.7625 - val_loss: 0.5744 - val_accuracy: 0.7575
Epoch 3/100
40/40 [==============================] - 6s 143ms/step - loss: 0.5681 - accuracy: 0.7707 - val_loss: 0.5724 - val_accuracy: 0.7559
Epoch 4/100
40/40 [==============================] - 6s 139ms/step - loss: 0.5560 - accuracy: 0.7840 - val_loss: 0.5577 - val_accuracy: 0.7591
Epoch 5/100
40/40 [==============================] - 6s 157ms/step - loss: 0.5421 - accuracy: 0.7910 - val_loss: 0.5939 - val_accuracy: 0.7433
Epoch 6/100
40/40 [==============================] - 6s 147ms/step - loss: 0.5506 - accuracy: 0.7840 - val_loss: 0.5290 - val_accuracy: 0.7765
Epoch 7/100
40/40 [==============================] - 7s 173ms/step - loss: 0.5442 - accuracy: 0.7914 - val_loss: 0.5533 - val_accuracy: 0.7718
Epoch 8/10

40/40 [==============================] - 7s 168ms/step - loss: 0.4786 - accuracy: 0.8363 - val_loss: 0.4488 - val_accuracy: 0.8304
Epoch 59/100
40/40 [==============================] - 7s 170ms/step - loss: 0.4615 - accuracy: 0.8465 - val_loss: 0.4601 - val_accuracy: 0.8209
Epoch 60/100
40/40 [==============================] - 7s 164ms/step - loss: 0.4680 - accuracy: 0.8418 - val_loss: 0.4422 - val_accuracy: 0.8368
Epoch 61/100
40/40 [==============================] - 6s 163ms/step - loss: 0.4743 - accuracy: 0.8379 - val_loss: 0.4439 - val_accuracy: 0.8352
Epoch 62/100
40/40 [==============================] - 8s 193ms/step - loss: 0.4754 - accuracy: 0.8398 - val_loss: 0.4446 - val_accuracy: 0.8352
Epoch 63/100
40/40 [==============================] - 7s 172ms/step - loss: 0.4684 - accuracy: 0.8402 - val_loss: 0.4429 - val_accuracy: 0.8352
Epoch 64/100
40/40 [==============================] - 7s 173ms/step - loss: 0.4701 - accuracy: 0.8387 - val_loss: 0.4414 - val_accuracy: 0.8320
Epoch

murmur: 84.47

In [17]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        out_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             out_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(out_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 9s 196ms/step - loss: 0.7684 - accuracy: 0.5203 - val_loss: 0.7211 - val_accuracy: 0.5166
Epoch 2/100
40/40 [==============================] - 7s 169ms/step - loss: 0.6819 - accuracy: 0.5566 - val_loss: 0.6605 - val_accuracy: 0.5880
Epoch 3/100
40/40 [==============================] - 7s 187ms/step - loss: 0.6850 - accuracy: 0.5512 - val_loss: 0.7485 - val_accuracy: 0.5309
Epoch 4/100
40/40 [==============================] - 7s 174ms/step - loss: 0.6799 - accuracy: 0.5699 - val_loss: 0.7703 - val_accuracy: 0.5341
Epoch 5/100
40/40 [==============================] - 7s 173ms/step - loss: 0.6711 - accuracy: 0.5754 - val_loss: 0.6601 - val_accuracy: 0.6054
Epoch 6/100
40/40 [==============================] - 7s 168ms/step - loss: 0.6770 - accuracy: 0.5762 - val_loss: 0.6872 - val_accuracy: 0.5800
Epoch 7/100
40/40 [==============================] - 7s 178ms/step - loss: 0.6724 - accuracy: 0.6027 - val_loss: 0.7407 - val_accuracy: 0.5642

Epoch 58/100
40/40 [==============================] - 7s 174ms/step - loss: 0.6266 - accuracy: 0.6645 - val_loss: 0.9854 - val_accuracy: 0.5895
Epoch 59/100
40/40 [==============================] - 7s 180ms/step - loss: 0.6269 - accuracy: 0.6680 - val_loss: 1.0600 - val_accuracy: 0.5769
Epoch 60/100
40/40 [==============================] - 7s 171ms/step - loss: 0.6195 - accuracy: 0.6676 - val_loss: 0.8962 - val_accuracy: 0.6133
Epoch 61/100
40/40 [==============================] - 7s 173ms/step - loss: 0.6159 - accuracy: 0.6809 - val_loss: 0.8795 - val_accuracy: 0.6165
Epoch 62/100
40/40 [==============================] - 7s 174ms/step - loss: 0.6236 - accuracy: 0.6734 - val_loss: 1.1513 - val_accuracy: 0.5769
Epoch 63/100
40/40 [==============================] - 7s 175ms/step - loss: 0.6257 - accuracy: 0.6668 - val_loss: 0.8945 - val_accuracy: 0.6149
Epoch 64/100
40/40 [==============================] - 7s 172ms/step - loss: 0.6220 - accuracy: 0.6672 - val_loss: 0.9771 - val_accuracy:

outcome: 62.76

**[팀코드 수정해야 할 함수들..]**

In [18]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10}

In [19]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [20]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    param_feature['model1'] = m_name1
    param_feature['model2'] = m_name2
    param_feature['model_fnm1'] = filename1
    param_feature['model_fnm2'] = filename2

    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [21]:
model_folder

'tmp_model0'

In [22]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'toy0_1_1', m_name2 = 'toy0_2_1', param_feature = params_feature)

1

In [23]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [24]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'toy0_1_1',
 'model2': 'toy0_2_1',
 'model_fnm1': 'tmp_model0/toy0_1_1_model1.hdf5',
 'model_fnm2': 'tmp_model0/toy0_2_1_model2.hdf5'}

In [38]:
def run_challenge_model(model, data, recordings, verbose):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'toy0_1_1' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = True, use_stft = True )
    if model['model2'] == 'toy0_2_1' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = True, use_stft = True)
       
    if model['model1'] == 'toy0_1_2' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = True, use_stft = False)
    if model['model2'] == 'toy0_2_2' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = True, use_stft = False)
 
    if model['model1'] == 'toy0_1_3' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model2'] == 'toy0_2_3' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
   
    if model['model1'] == 'toy0_1_4' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = True, use_stft = True)
    if model['model2'] == 'toy0_2_4' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = True, use_stft = True)

    if model['model1'] == 'toy0_1_5' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = True, use_stft = False)
    if model['model2'] == 'toy0_2_5' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = True, use_stft = False)
    
    if model['model1'] == 'toy0_1_6' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = False, use_stft = True)
    if model['model2'] == 'toy0_2_6' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = False, use_stft = True)
   
    
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    
#    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
        features['mel1'].append(mel1)
    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
    features['mel1'] = np.array(features['mel1'])

    features['cqt1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_cqt(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
        features['cqt1'].append(mel1)
    M, N = features['cqt1'][0].shape
    for i in range(len(features['cqt1'])) :
        features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    features['cqt1'] = np.array(features['cqt1'])

    features['stft1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                       win_length = win_length)[0]
        features['stft1'].append(mel1)
    M, N = features['stft1'][0].shape
    for i in range(len(features['stft1'])) :
        features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])

    # Get classifier probabilities.
    idx1 = res1.argmax(axis=0)[0]
    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

**[대회 평가용 run_model 함수 (수정 불필요)]**

In [39]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [40]:
## 우리 모형 저장된 폴더이름
model_folder

'tmp_model0'

In [41]:
## 테스트 데이터 폴더
test_folder

'/Data2/hmd/data_split/murmur/test'

In [42]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/Data2/hmd/hmd_sy/notebooks/out_model0'

In [43]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...
Done.


In [44]:
!cat /Data2/hmd/hmd_sy/notebooks/out_resmax1/49628.csv

#49628
Present,Unknown,Absent,Abnormal,Normal
1,0,0,1,0
0.65907294,0.0066230143,0.334304,0.75112474,0.24887528


In [45]:
from evaluate_model import *

### 대회 평가 지표

In [46]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.785,0.581,0.501,0.791,0.687,17764.311

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.676,0.679,0.518,0.581,0.334,21317.120

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.845,0.682,0.829
AUPRC,0.725,0.125,0.893
F-measure,0.627,0.000,0.877
Accuracy,0.684,0.000,0.899

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.676,0.676
AUPRC,0.729,0.630
F-measure,0.344,0.692
Accuracy,0.214,0.968



### Threshold 변경해가며 결과 탐색

In [47]:
test_folder

'/Data2/hmd/data_split/murmur/test'

In [48]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.3587643012460733
0.6270053398445027
0.22504243906282723
0.7749575600507853


In [49]:
## threshold 바꿔가면서 결과 출력

for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")


threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.785,0.581,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.676,0.679,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.845,0.682,0.829
AUPRC,0.725,0.125,0.893
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.676,0.676
AUPRC,0.729,0.630
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.785,0.581,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.676,0.679,0.432,0.539,0.815,13264.157

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.845,0.682,0.829
AUPRC,0.725,0.125,0.893
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU

threshold:  0.8
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.785,0.581,0.509,0.817,0.636,20898.542

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.676,0.679,0.422,0.534,0.237,23819.581

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.845,0.682,0.829
AUPRC,0.725,0.125,0.893
F-measure,0.635,0.000,0.892
Accuracy,0.526,0.000,0.978

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.676,0.676
AUPRC,0.729,0.630
F-measure,0.168,0.676
Accuracy,0.092,1.000

-------------


## mel, cqt 

In [52]:
model_folder = 'tmp_model0_2'

In [53]:
model1 = get_toy0_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = False)
model2 = get_toy0_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = True, use_stft = False)


In [54]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        mm_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             mm_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 8s 195ms/step - loss: 0.7187 - accuracy: 0.7457 - val_loss: 0.6111 - val_accuracy: 0.7448
Epoch 2/100
40/40 [==============================] - 6s 163ms/step - loss: 0.5899 - accuracy: 0.7676 - val_loss: 0.5881 - val_accuracy: 0.7448
Epoch 3/100
40/40 [==============================] - 6s 162ms/step - loss: 0.5849 - accuracy: 0.7742 - val_loss: 0.5922 - val_accuracy: 0.7448
Epoch 4/100
40/40 [==============================] - 7s 174ms/step - loss: 0.5824 - accuracy: 0.7656 - val_loss: 0.5780 - val_accuracy: 0.7448
Epoch 5/100
40/40 [==============================] - 7s 169ms/step - loss: 0.5863 - accuracy: 0.7668 - val_loss: 0.5768 - val_accuracy: 0.7496
Epoch 6/100
40/40 [==============================] - 6s 161ms/step - loss: 0.5892 - accuracy: 0.7590 - val_loss: 0.5809 - val_accuracy: 0.7655
Epoch 7/100
40/40 [==============================] - 7s 176ms/step - loss: 0.5889 - accuracy: 0.7602 - val_loss: 0.5813 - val_accuracy: 0.7655

Epoch 58/100
40/40 [==============================] - 7s 166ms/step - loss: 0.5419 - accuracy: 0.7809 - val_loss: 0.4953 - val_accuracy: 0.8067
Epoch 59/100
40/40 [==============================] - 6s 161ms/step - loss: 0.5514 - accuracy: 0.7715 - val_loss: 0.4943 - val_accuracy: 0.8098
Epoch 60/100
40/40 [==============================] - 7s 169ms/step - loss: 0.5483 - accuracy: 0.7730 - val_loss: 0.4981 - val_accuracy: 0.8082
Epoch 61/100
40/40 [==============================] - 6s 160ms/step - loss: 0.5341 - accuracy: 0.7848 - val_loss: 0.4882 - val_accuracy: 0.8177
Epoch 62/100
40/40 [==============================] - 6s 161ms/step - loss: 0.5407 - accuracy: 0.7922 - val_loss: 0.4971 - val_accuracy: 0.8067
Epoch 63/100
40/40 [==============================] - 7s 183ms/step - loss: 0.5453 - accuracy: 0.7660 - val_loss: 0.4912 - val_accuracy: 0.8067
Epoch 64/100
40/40 [==============================] - 6s 165ms/step - loss: 0.5538 - accuracy: 0.7703 - val_loss: 0.4881 - val_accuracy:

murmur : 81.93

In [55]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        out_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             out_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(out_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 8s 186ms/step - loss: 0.8998 - accuracy: 0.5082 - val_loss: 0.6876 - val_accuracy: 0.5024
Epoch 2/100
40/40 [==============================] - 7s 170ms/step - loss: 0.6953 - accuracy: 0.5055 - val_loss: 0.6894 - val_accuracy: 0.5404
Epoch 3/100
40/40 [==============================] - 7s 171ms/step - loss: 0.6926 - accuracy: 0.5070 - val_loss: 0.6870 - val_accuracy: 0.6197
Epoch 4/100
40/40 [==============================] - 6s 163ms/step - loss: 0.6927 - accuracy: 0.5172 - val_loss: 0.6897 - val_accuracy: 0.5040
Epoch 5/100
40/40 [==============================] - 7s 166ms/step - loss: 0.6938 - accuracy: 0.5164 - val_loss: 0.6914 - val_accuracy: 0.4992
Epoch 6/100
40/40 [==============================] - 7s 168ms/step - loss: 0.6906 - accuracy: 0.5270 - val_loss: 0.6877 - val_accuracy: 0.4992
Epoch 7/100
40/40 [==============================] - 6s 157ms/step - loss: 0.6928 - accuracy: 0.5211 - val_loss: 0.6779 - val_accuracy: 0.5531

Epoch 58/100
40/40 [==============================] - 6s 161ms/step - loss: 0.6842 - accuracy: 0.5516 - val_loss: 0.6480 - val_accuracy: 0.6371
Epoch 59/100
40/40 [==============================] - 6s 154ms/step - loss: 0.6804 - accuracy: 0.5754 - val_loss: 0.6469 - val_accuracy: 0.6339
Epoch 60/100
40/40 [==============================] - 7s 166ms/step - loss: 0.6838 - accuracy: 0.5684 - val_loss: 0.6547 - val_accuracy: 0.6244
Epoch 61/100
40/40 [==============================] - 7s 168ms/step - loss: 0.6829 - accuracy: 0.5609 - val_loss: 0.6485 - val_accuracy: 0.6323
Epoch 62/100
40/40 [==============================] - 7s 176ms/step - loss: 0.6821 - accuracy: 0.5691 - val_loss: 0.6537 - val_accuracy: 0.6197
Epoch 63/100
40/40 [==============================] - 6s 165ms/step - loss: 0.6797 - accuracy: 0.5801 - val_loss: 0.6524 - val_accuracy: 0.6244
Epoch 64/100
40/40 [==============================] - 6s 158ms/step - loss: 0.6802 - accuracy: 0.5668 - val_loss: 0.6505 - val_accuracy:

outcome: 65.29

In [56]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'toy0_1_1',
 'model2': 'toy0_2_1',
 'model_fnm1': 'tmp_model0/toy0_1_1_model1.hdf5',
 'model_fnm2': 'tmp_model0/toy0_2_1_model2.hdf5'}

In [57]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [58]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'toy0_1_2', m_name2 = 'toy0_2_2', param_feature = params_feature)

1

In [60]:
def run_challenge_model(model, data, recordings, verbose):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'toy0_1_1' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = True, use_stft = True )
    if model['model2'] == 'toy0_2_1' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = True, use_stft = True)
       
    if model['model1'] == 'toy0_1_2' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = True, use_stft = False)
    if model['model2'] == 'toy0_2_2' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = True, use_stft = False)
 
    if model['model1'] == 'toy0_1_3' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
    if model['model2'] == 'toy0_2_3' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = True, use_cqt = False, use_stft = False)
   
    if model['model1'] == 'toy0_1_4' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = True, use_stft = True)
    if model['model2'] == 'toy0_2_4' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = True, use_stft = True)

    if model['model1'] == 'toy0_1_5' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = True, use_stft = False)
    if model['model2'] == 'toy0_2_5' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = True, use_stft = False)
    
    if model['model1'] == 'toy0_1_6' :
        model1 = get_toy0_1(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = False, use_stft = True)
    if model['model2'] == 'toy0_2_6' :
        model2 = get_toy0_2(model['mel_shape'],model['cqt_shape'],model['stft_shape'], use_mel = False, use_cqt = False, use_stft = True)
   
    
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    
#    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
        features['mel1'].append(mel1)
    M, N = features['mel1'][0].shape
    for i in range(len(features['mel1'])) :
        features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
    features['mel1'] = np.array(features['mel1'])

    features['cqt1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_cqt(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
        features['cqt1'].append(mel1)
    M, N = features['cqt1'][0].shape
    for i in range(len(features['cqt1'])) :
        features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    features['cqt1'] = np.array(features['cqt1'])

    features['stft1'] = []
    for i in range(len(recordings)) :
        mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                       win_length = win_length)[0]
        features['stft1'].append(mel1)
    M, N = features['stft1'][0].shape
    for i in range(len(features['stft1'])) :
        features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], features['loc'], features['mel1'], features['cqt1'], features['stft1']])

    # Get classifier probabilities.
    idx1 = res1.argmax(axis=0)[0]
    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

In [61]:
## 우리 모형 저장된 폴더이름
model_folder

'tmp_model0_2'

In [62]:
## 테스트 데이터 폴더
test_folder

'/Data2/hmd/data_split/murmur/test'

In [63]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/Data2/hmd/hmd_sy/notebooks/out_model0_2'

In [64]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...
Done.


### 대회 평가 지표

In [65]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.741,0.562,0.512,0.822,0.628,21525.500

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.665,0.661,0.651,0.660,0.545,15714.837

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.793,0.631,0.797
AUPRC,0.696,0.105,0.884
F-measure,0.644,0.000,0.893
Accuracy,0.500,0.000,0.993

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.665,0.665
AUPRC,0.736,0.586
F-measure,0.596,0.706
Accuracy,0.490,0.839



In [66]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.2805060958638743
0.702131920209424
0.501968787591623
0.49803121492146596


### threshold 바꿔가면서 결과 출력

In [67]:
for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")


threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.741,0.562,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.665,0.661,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.793,0.631,0.797
AUPRC,0.696,0.105,0.884
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.665,0.665
AUPRC,0.736,0.586
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.741,0.562,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.665,0.661,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.793,0.631,0.797
AUPRC,0.696,0.105,0.884
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU

## mel

In [68]:
model_folder = 'tmp_model0_3'

In [69]:
model1 = get_toy0_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = False, use_stft = False)
model2 = get_toy0_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = True, use_cqt = False, use_stft = False)


In [70]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        mm_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             mm_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 8s 186ms/step - loss: 0.7990 - accuracy: 0.7379 - val_loss: 0.6002 - val_accuracy: 0.7448
Epoch 2/100
40/40 [==============================] - 6s 163ms/step - loss: 0.5858 - accuracy: 0.7699 - val_loss: 0.6008 - val_accuracy: 0.7448
Epoch 3/100
40/40 [==============================] - 6s 164ms/step - loss: 0.5934 - accuracy: 0.7605 - val_loss: 0.6004 - val_accuracy: 0.7448
Epoch 4/100
40/40 [==============================] - 6s 155ms/step - loss: 0.5726 - accuracy: 0.7797 - val_loss: 0.5786 - val_accuracy: 0.7544
Epoch 5/100
40/40 [==============================] - 6s 159ms/step - loss: 0.5780 - accuracy: 0.7715 - val_loss: 0.5763 - val_accuracy: 0.7512
Epoch 6/100
40/40 [==============================] - 6s 161ms/step - loss: 0.5874 - accuracy: 0.7605 - val_loss: 0.5846 - val_accuracy: 0.7480
Epoch 7/100
40/40 [==============================] - 6s 161ms/step - loss: 0.5766 - accuracy: 0.7715 - val_loss: 0.5633 - val_accuracy: 0.7544

Epoch 58/100
40/40 [==============================] - 6s 145ms/step - loss: 0.5455 - accuracy: 0.7766 - val_loss: 0.4968 - val_accuracy: 0.8082
Epoch 59/100
40/40 [==============================] - 6s 145ms/step - loss: 0.5609 - accuracy: 0.7699 - val_loss: 0.5041 - val_accuracy: 0.8114
Epoch 60/100
40/40 [==============================] - 6s 145ms/step - loss: 0.5362 - accuracy: 0.7852 - val_loss: 0.4997 - val_accuracy: 0.8067
Epoch 61/100
40/40 [==============================] - 6s 143ms/step - loss: 0.5471 - accuracy: 0.7703 - val_loss: 0.4947 - val_accuracy: 0.8098
Epoch 62/100
40/40 [==============================] - 6s 146ms/step - loss: 0.5454 - accuracy: 0.7746 - val_loss: 0.4969 - val_accuracy: 0.8035
Epoch 63/100
40/40 [==============================] - 6s 153ms/step - loss: 0.5408 - accuracy: 0.7777 - val_loss: 0.5020 - val_accuracy: 0.8162
Epoch 64/100
40/40 [==============================] - 7s 167ms/step - loss: 0.5600 - accuracy: 0.7707 - val_loss: 0.4972 - val_accuracy:

mel 성능 별로 안좋음

murmur: 81.62

In [71]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        out_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             out_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(out_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 7s 177ms/step - loss: 0.7620 - accuracy: 0.5000 - val_loss: 0.6853 - val_accuracy: 0.5578
Epoch 2/100
40/40 [==============================] - 6s 161ms/step - loss: 0.6912 - accuracy: 0.5230 - val_loss: 0.6714 - val_accuracy: 0.5895
Epoch 3/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6922 - accuracy: 0.5145 - val_loss: 0.6765 - val_accuracy: 0.6197
Epoch 4/100
40/40 [==============================] - 6s 156ms/step - loss: 0.6916 - accuracy: 0.5246 - val_loss: 0.6784 - val_accuracy: 0.6117
Epoch 5/100
40/40 [==============================] - 6s 165ms/step - loss: 0.6902 - accuracy: 0.5324 - val_loss: 0.6735 - val_accuracy: 0.5705
Epoch 6/100
40/40 [==============================] - 7s 171ms/step - loss: 0.6903 - accuracy: 0.5383 - val_loss: 0.6768 - val_accuracy: 0.5911
Epoch 7/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6907 - accuracy: 0.5305 - val_loss: 0.6754 - val_accuracy: 0.5864

Epoch 58/100
40/40 [==============================] - 6s 153ms/step - loss: 0.6824 - accuracy: 0.5621 - val_loss: 0.6502 - val_accuracy: 0.6466
Epoch 59/100
40/40 [==============================] - 6s 153ms/step - loss: 0.6809 - accuracy: 0.5730 - val_loss: 0.6524 - val_accuracy: 0.6418
Epoch 60/100
40/40 [==============================] - 6s 153ms/step - loss: 0.6787 - accuracy: 0.5844 - val_loss: 0.6498 - val_accuracy: 0.6529
Epoch 61/100
40/40 [==============================] - 6s 150ms/step - loss: 0.6799 - accuracy: 0.5660 - val_loss: 0.6546 - val_accuracy: 0.6498
Epoch 62/100
40/40 [==============================] - 6s 154ms/step - loss: 0.6801 - accuracy: 0.5703 - val_loss: 0.6609 - val_accuracy: 0.6260
Epoch 63/100
40/40 [==============================] - 6s 150ms/step - loss: 0.6787 - accuracy: 0.5750 - val_loss: 0.6400 - val_accuracy: 0.6466
Epoch 64/100
40/40 [==============================] - 6s 153ms/step - loss: 0.6761 - accuracy: 0.5777 - val_loss: 0.6420 - val_accuracy:

outcome의 경우 mel feature만 쓴 경우 성능 좋음

outcome: 64.18

In [72]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'toy0_1_2',
 'model2': 'toy0_2_2',
 'model_fnm1': 'tmp_model0_2/toy0_1_2_model1.hdf5',
 'model_fnm2': 'tmp_model0_2/toy0_2_2_model2.hdf5'}

In [73]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [74]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'toy0_1_3', m_name2 = 'toy0_2_3', param_feature = params_feature)

1

In [75]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'toy0_1_3',
 'model2': 'toy0_2_3',
 'model_fnm1': 'tmp_model0_3/toy0_1_3_model1.hdf5',
 'model_fnm2': 'tmp_model0_3/toy0_2_3_model2.hdf5'}

In [76]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/Data2/hmd/hmd_sy/notebooks/out_model0_3'

In [77]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...
Done.


### 대회 평가 지표

In [78]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.754,0.561,0.515,0.827,0.631,21316.076

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.719,0.703,0.670,0.675,0.578,14904.459

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.786,0.676,0.799
AUPRC,0.670,0.117,0.897
F-measure,0.644,0.000,0.900
Accuracy,0.500,0.000,1.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.719,0.719
AUPRC,0.776,0.630
F-measure,0.627,0.713
Accuracy,0.531,0.828



In [79]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.2976153259947644
0.6940745074345549
0.49772573722513086
0.5022742619895287


### threshold 바꿔가면서 결과 출력

In [80]:
## threshold 바꿔가면서 결과 출력

for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")


threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.754,0.561,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.719,0.703,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.786,0.676,0.799
AUPRC,0.670,0.117,0.897
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.719,0.719
AUPRC,0.776,0.630
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.754,0.561,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.719,0.703,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.786,0.676,0.799
AUPRC,0.670,0.117,0.897
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU

## cqt, stft

In [81]:
model_folder = 'tmp_model0_4'

In [82]:
model1 = get_toy0_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = False, use_cqt = True, use_stft = True)
model2 = get_toy0_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = False, use_cqt = True, use_stft = True)


In [83]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        mm_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             mm_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 9s 207ms/step - loss: 0.6291 - accuracy: 0.7574 - val_loss: 0.5683 - val_accuracy: 0.7464
Epoch 2/100
40/40 [==============================] - 6s 159ms/step - loss: 0.5596 - accuracy: 0.7805 - val_loss: 0.5614 - val_accuracy: 0.7528
Epoch 3/100
40/40 [==============================] - 6s 150ms/step - loss: 0.5512 - accuracy: 0.7863 - val_loss: 0.5652 - val_accuracy: 0.7559
Epoch 4/100
40/40 [==============================] - 7s 166ms/step - loss: 0.5518 - accuracy: 0.7836 - val_loss: 0.5700 - val_accuracy: 0.7591
Epoch 5/100
40/40 [==============================] - 6s 157ms/step - loss: 0.5564 - accuracy: 0.7758 - val_loss: 0.5336 - val_accuracy: 0.7718
Epoch 6/100
40/40 [==============================] - 6s 156ms/step - loss: 0.5374 - accuracy: 0.8078 - val_loss: 0.5296 - val_accuracy: 0.7908
Epoch 7/100
40/40 [==============================] - 6s 155ms/step - loss: 0.5421 - accuracy: 0.8000 - val_loss: 0.5921 - val_accuracy: 0.7655

Epoch 58/100
40/40 [==============================] - 6s 162ms/step - loss: 0.4531 - accuracy: 0.8469 - val_loss: 0.4329 - val_accuracy: 0.8384
Epoch 59/100
40/40 [==============================] - 6s 159ms/step - loss: 0.4479 - accuracy: 0.8547 - val_loss: 0.4292 - val_accuracy: 0.8384
Epoch 60/100
40/40 [==============================] - 6s 160ms/step - loss: 0.4640 - accuracy: 0.8379 - val_loss: 0.4399 - val_accuracy: 0.8304
Epoch 61/100
40/40 [==============================] - 6s 161ms/step - loss: 0.4590 - accuracy: 0.8477 - val_loss: 0.4173 - val_accuracy: 0.8399
Epoch 62/100
40/40 [==============================] - 7s 167ms/step - loss: 0.4493 - accuracy: 0.8578 - val_loss: 0.4174 - val_accuracy: 0.8399
Epoch 63/100
40/40 [==============================] - 6s 159ms/step - loss: 0.4591 - accuracy: 0.8469 - val_loss: 0.4194 - val_accuracy: 0.8415
Epoch 64/100
40/40 [==============================] - 6s 164ms/step - loss: 0.4559 - accuracy: 0.8406 - val_loss: 0.4167 - val_accuracy:

murmur: 83.66

In [84]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        out_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             out_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(out_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 7s 174ms/step - loss: 0.7210 - accuracy: 0.5039 - val_loss: 0.6730 - val_accuracy: 0.6070
Epoch 2/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6863 - accuracy: 0.5301 - val_loss: 0.6657 - val_accuracy: 0.5880
Epoch 3/100
40/40 [==============================] - 6s 157ms/step - loss: 0.6860 - accuracy: 0.5410 - val_loss: 0.6706 - val_accuracy: 0.5055
Epoch 4/100
40/40 [==============================] - 6s 164ms/step - loss: 0.6791 - accuracy: 0.5832 - val_loss: 0.6449 - val_accuracy: 0.6482
Epoch 5/100
40/40 [==============================] - 7s 168ms/step - loss: 0.6793 - accuracy: 0.5652 - val_loss: 0.6501 - val_accuracy: 0.6371
Epoch 6/100
40/40 [==============================] - 6s 163ms/step - loss: 0.6801 - accuracy: 0.5703 - val_loss: 0.6697 - val_accuracy: 0.5911
Epoch 7/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6786 - accuracy: 0.5809 - val_loss: 0.6487 - val_accuracy: 0.6323

Epoch 58/100
40/40 [==============================] - 7s 165ms/step - loss: 0.6406 - accuracy: 0.6375 - val_loss: 0.7604 - val_accuracy: 0.5626
Epoch 59/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6372 - accuracy: 0.6391 - val_loss: 0.7577 - val_accuracy: 0.4945
Epoch 60/100
40/40 [==============================] - 7s 170ms/step - loss: 0.6277 - accuracy: 0.6523 - val_loss: 0.7675 - val_accuracy: 0.4881
Epoch 61/100
40/40 [==============================] - 6s 163ms/step - loss: 0.6308 - accuracy: 0.6680 - val_loss: 0.7876 - val_accuracy: 0.5341
Epoch 62/100
40/40 [==============================] - 6s 163ms/step - loss: 0.6331 - accuracy: 0.6449 - val_loss: 0.7905 - val_accuracy: 0.5119
Epoch 63/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6365 - accuracy: 0.6457 - val_loss: 0.7976 - val_accuracy: 0.4897
Epoch 64/100
40/40 [==============================] - 7s 168ms/step - loss: 0.6293 - accuracy: 0.6605 - val_loss: 0.7488 - val_accuracy:

cqt, stft outcome 성능 별로

outcmoe : 61.49

In [85]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'toy0_1_3',
 'model2': 'toy0_2_3',
 'model_fnm1': 'tmp_model0_3/toy0_1_3_model1.hdf5',
 'model_fnm2': 'tmp_model0_3/toy0_2_3_model2.hdf5'}

In [86]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [87]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'toy0_1_4', m_name2 = 'toy0_2_4', param_feature = params_feature)

1

In [88]:
## 우리 모형 저장된 폴더이름
model_folder

'tmp_model0_4'

In [89]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/Data2/hmd/hmd_sy/notebooks/out_model0_4'

In [90]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...
Done.


### 대회 평가 지표

In [91]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.787,0.618,0.500,0.785,0.695,17558.235

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.581,0.595,0.508,0.513,0.573,14350.874

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.872,0.631,0.857
AUPRC,0.763,0.178,0.913
F-measure,0.628,0.000,0.872
Accuracy,0.711,0.000,0.885

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.581,0.581
AUPRC,0.677,0.512
F-measure,0.559,0.456
Accuracy,0.602,0.419



In [92]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.3429400869371727
0.6420621116188482
0.5510707032984292
0.4489292904727748


### threshold 바꿔가면서 결과 출력

In [93]:
## threshold 바꿔가면서 결과 출력

for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")


threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.787,0.618,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.581,0.595,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.872,0.631,0.857
AUPRC,0.763,0.178,0.913
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.581,0.581
AUPRC,0.677,0.512
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.787,0.618,0.131,0.220,0.523,14915.104

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.581,0.595,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.872,0.631,0.857
AUPRC,0.763,0.178,0.913
F-measure,0.338,0.000,0.056
Accuracy,1.000,0.000,0.029

#Outcome scores (per class)
Classes,Abnormal,Normal
AU

threshold:  0.8
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.787,0.618,0.508,0.817,0.625,21525.892

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.581,0.595,0.485,0.560,0.300,22153.332

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.872,0.631,0.857
AUPRC,0.763,0.178,0.913
F-measure,0.633,0.000,0.890
Accuracy,0.500,0.000,0.986

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.581,0.581
AUPRC,0.677,0.512
F-measure,0.288,0.682
Accuracy,0.173,0.968

-------------


## cqt

In [94]:
model_folder = 'tmp_model0_5'

In [95]:
model1 = get_toy0_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = False, use_cqt = True, use_stft = False)
model2 = get_toy0_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = False, use_cqt = True, use_stft = False)


In [96]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        mm_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             mm_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 8s 193ms/step - loss: 0.6211 - accuracy: 0.7598 - val_loss: 0.6209 - val_accuracy: 0.7448
Epoch 2/100
40/40 [==============================] - 6s 157ms/step - loss: 0.5935 - accuracy: 0.7707 - val_loss: 0.6191 - val_accuracy: 0.7448
Epoch 3/100
40/40 [==============================] - 6s 164ms/step - loss: 0.5990 - accuracy: 0.7559 - val_loss: 0.6181 - val_accuracy: 0.7448
Epoch 4/100
40/40 [==============================] - 6s 153ms/step - loss: 0.5924 - accuracy: 0.7695 - val_loss: 0.6183 - val_accuracy: 0.7448
Epoch 5/100
40/40 [==============================] - 6s 154ms/step - loss: 0.5896 - accuracy: 0.7723 - val_loss: 0.6191 - val_accuracy: 0.7448
Epoch 6/100
40/40 [==============================] - 6s 146ms/step - loss: 0.5937 - accuracy: 0.7656 - val_loss: 0.6308 - val_accuracy: 0.7448
Epoch 7/100
40/40 [==============================] - 6s 152ms/step - loss: 0.5944 - accuracy: 0.7660 - val_loss: 0.6169 - val_accuracy: 0.7448

Epoch 58/100
40/40 [==============================] - 6s 150ms/step - loss: 0.5937 - accuracy: 0.7602 - val_loss: 0.6084 - val_accuracy: 0.7448
Epoch 59/100
40/40 [==============================] - 7s 166ms/step - loss: 0.5983 - accuracy: 0.7535 - val_loss: 0.6090 - val_accuracy: 0.7448
Epoch 60/100
40/40 [==============================] - 6s 163ms/step - loss: 0.5844 - accuracy: 0.7691 - val_loss: 0.6075 - val_accuracy: 0.7448
Epoch 61/100
40/40 [==============================] - 6s 157ms/step - loss: 0.5770 - accuracy: 0.7746 - val_loss: 0.6097 - val_accuracy: 0.7448
Epoch 62/100
40/40 [==============================] - 6s 152ms/step - loss: 0.5832 - accuracy: 0.7691 - val_loss: 0.6081 - val_accuracy: 0.7448
Epoch 63/100
40/40 [==============================] - 6s 154ms/step - loss: 0.5901 - accuracy: 0.7625 - val_loss: 0.6071 - val_accuracy: 0.7448
Epoch 64/100
40/40 [==============================] - 6s 165ms/step - loss: 0.5823 - accuracy: 0.7660 - val_loss: 0.6086 - val_accuracy:

In [97]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        out_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             out_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(out_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 7s 179ms/step - loss: 0.7023 - accuracy: 0.5156 - val_loss: 0.6965 - val_accuracy: 0.5008
Epoch 2/100
40/40 [==============================] - 7s 170ms/step - loss: 0.6939 - accuracy: 0.5074 - val_loss: 0.6955 - val_accuracy: 0.5008
Epoch 3/100
40/40 [==============================] - 7s 169ms/step - loss: 0.6942 - accuracy: 0.5012 - val_loss: 0.6946 - val_accuracy: 0.5008
Epoch 4/100
40/40 [==============================] - 6s 155ms/step - loss: 0.6925 - accuracy: 0.5309 - val_loss: 0.6932 - val_accuracy: 0.5008
Epoch 5/100
40/40 [==============================] - 6s 164ms/step - loss: 0.6921 - accuracy: 0.5227 - val_loss: 0.6943 - val_accuracy: 0.5008
Epoch 6/100
40/40 [==============================] - 7s 169ms/step - loss: 0.6931 - accuracy: 0.5117 - val_loss: 0.6940 - val_accuracy: 0.5008
Epoch 7/100
40/40 [==============================] - 7s 174ms/step - loss: 0.6923 - accuracy: 0.5266 - val_loss: 0.6937 - val_accuracy: 0.5008

Epoch 58/100
40/40 [==============================] - 7s 171ms/step - loss: 0.6920 - accuracy: 0.5211 - val_loss: 0.6940 - val_accuracy: 0.5008
Epoch 59/100
40/40 [==============================] - 7s 170ms/step - loss: 0.6921 - accuracy: 0.5168 - val_loss: 0.6939 - val_accuracy: 0.5008
Epoch 60/100
40/40 [==============================] - 7s 169ms/step - loss: 0.6927 - accuracy: 0.5168 - val_loss: 0.6939 - val_accuracy: 0.5008
Epoch 61/100
40/40 [==============================] - 6s 162ms/step - loss: 0.6923 - accuracy: 0.5211 - val_loss: 0.6939 - val_accuracy: 0.5008
Epoch 62/100
40/40 [==============================] - 6s 161ms/step - loss: 0.6922 - accuracy: 0.5234 - val_loss: 0.6939 - val_accuracy: 0.5008
Epoch 63/100
40/40 [==============================] - 6s 153ms/step - loss: 0.6922 - accuracy: 0.5176 - val_loss: 0.6939 - val_accuracy: 0.5008
Epoch 64/100
40/40 [==============================] - 7s 166ms/step - loss: 0.6926 - accuracy: 0.5129 - val_loss: 0.6939 - val_accuracy:

In [98]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [99]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'toy0_1_5', m_name2 = 'toy0_2_5', param_feature = params_feature)

1

In [100]:
## 우리 모형 저장된 폴더이름
model_folder

'tmp_model0_5'

In [101]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/Data2/hmd/hmd_sy/notebooks/out_model0_5'

In [102]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...
Done.


### 대회 평가 지표

In [103]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.577,0.425,0.281,0.728,0.375,25689.450

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.349,0.407,0.327,0.487,0.160,25689.450

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.673,0.438,0.619
AUPRC,0.438,0.065,0.773
F-measure,0.000,0.000,0.842
Accuracy,0.000,0.000,1.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.349,0.349
AUPRC,0.424,0.389
F-measure,0.000,0.655
Accuracy,0.000,1.000



In [104]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.22222438654450263
0.7474463609424083
0.48091128502617797
0.519088717434555


### threshold 바꿔가면서 결과 출력

In [105]:
## threshold 바꿔가면서 결과 출력

for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")


threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.577,0.425,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.349,0.407,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.673,0.438,0.619
AUPRC,0.438,0.065,0.773
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.349,0.349
AUPRC,0.424,0.389
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.577,0.425,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.349,0.407,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.673,0.438,0.619
AUPRC,0.438,0.065,0.773
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AU

## stft

In [106]:
model_folder = 'tmp_model0_6'

In [107]:
model1 = get_toy0_1(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = False, use_cqt = False, use_stft = True)
model2 = get_toy0_2(mel_input_shape, cqt_input_shape, stft_input_shape, use_mel = False, use_cqt = False, use_stft = True)


In [108]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
#          'highpass': [.5, [78,79,80,81,82,83,84,85]]
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        mm_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        mm_lbs_test,  ## our Y
#                        **params_no_shuffle)()

    
model1.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             mm_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(mm_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 8s 194ms/step - loss: 0.6026 - accuracy: 0.7715 - val_loss: 0.6036 - val_accuracy: 0.7448
Epoch 2/100
40/40 [==============================] - 6s 157ms/step - loss: 0.5728 - accuracy: 0.7793 - val_loss: 0.5841 - val_accuracy: 0.7544
Epoch 3/100
40/40 [==============================] - 6s 152ms/step - loss: 0.5738 - accuracy: 0.7750 - val_loss: 0.6318 - val_accuracy: 0.7448
Epoch 4/100
40/40 [==============================] - 6s 154ms/step - loss: 0.5586 - accuracy: 0.7824 - val_loss: 0.5512 - val_accuracy: 0.7718
Epoch 5/100
40/40 [==============================] - 6s 153ms/step - loss: 0.5524 - accuracy: 0.7824 - val_loss: 0.5481 - val_accuracy: 0.7655
Epoch 6/100
40/40 [==============================] - 6s 163ms/step - loss: 0.5401 - accuracy: 0.8031 - val_loss: 0.5233 - val_accuracy: 0.7892
Epoch 7/100
40/40 [==============================] - 6s 159ms/step - loss: 0.5377 - accuracy: 0.7973 - val_loss: 0.5242 - val_accuracy: 0.7987

Epoch 58/100
40/40 [==============================] - 6s 155ms/step - loss: 0.4607 - accuracy: 0.8449 - val_loss: 0.4364 - val_accuracy: 0.8447
Epoch 59/100
40/40 [==============================] - 6s 162ms/step - loss: 0.4566 - accuracy: 0.8457 - val_loss: 0.4374 - val_accuracy: 0.8368
Epoch 60/100
40/40 [==============================] - 6s 158ms/step - loss: 0.4660 - accuracy: 0.8387 - val_loss: 0.4430 - val_accuracy: 0.8336
Epoch 61/100
40/40 [==============================] - 6s 164ms/step - loss: 0.4505 - accuracy: 0.8543 - val_loss: 0.4397 - val_accuracy: 0.8320
Epoch 62/100
40/40 [==============================] - 6s 154ms/step - loss: 0.4559 - accuracy: 0.8477 - val_loss: 0.4365 - val_accuracy: 0.8352
Epoch 63/100
40/40 [==============================] - 6s 163ms/step - loss: 0.4538 - accuracy: 0.8555 - val_loss: 0.4306 - val_accuracy: 0.8463
Epoch 64/100
40/40 [==============================] - 6s 158ms/step - loss: 0.4594 - accuracy: 0.8484 - val_loss: 0.4326 - val_accuracy:

stft 성능 괜찮음  
murmur: 84.15

In [109]:
n_epoch = 100
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))
batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
#          'lowpass': [.5, [11,12,13,14,15,16,17,18]]
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
          'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'], 
           features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
                        out_lbs_trn,  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()

    
model2.fit(TrainDGen_1,
          validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'], features_test['mel1'], 
                              features_test['cqt1'], features_test['stft1']], 
                             out_lbs_test), 
                             callbacks=[lr],
                              steps_per_epoch=np.ceil(len(out_lbs_trn)/64),
                             epochs = n_epoch)

Epoch 1/100
40/40 [==============================] - 8s 182ms/step - loss: 0.7026 - accuracy: 0.5355 - val_loss: 0.6913 - val_accuracy: 0.5658
Epoch 2/100
40/40 [==============================] - 6s 163ms/step - loss: 0.6808 - accuracy: 0.5426 - val_loss: 0.7017 - val_accuracy: 0.5832
Epoch 3/100
40/40 [==============================] - 7s 173ms/step - loss: 0.6841 - accuracy: 0.5512 - val_loss: 0.6737 - val_accuracy: 0.5689
Epoch 4/100
40/40 [==============================] - 7s 178ms/step - loss: 0.6832 - accuracy: 0.5484 - val_loss: 0.6485 - val_accuracy: 0.6212
Epoch 5/100
40/40 [==============================] - 7s 167ms/step - loss: 0.6800 - accuracy: 0.5598 - val_loss: 0.6664 - val_accuracy: 0.5990
Epoch 6/100
40/40 [==============================] - 6s 163ms/step - loss: 0.6767 - accuracy: 0.5805 - val_loss: 0.6478 - val_accuracy: 0.6165
Epoch 7/100
40/40 [==============================] - 7s 176ms/step - loss: 0.6749 - accuracy: 0.5797 - val_loss: 0.6239 - val_accuracy: 0.6498

Epoch 58/100
40/40 [==============================] - 7s 185ms/step - loss: 0.6243 - accuracy: 0.6602 - val_loss: 0.9680 - val_accuracy: 0.6006
Epoch 59/100
40/40 [==============================] - 7s 168ms/step - loss: 0.6308 - accuracy: 0.6566 - val_loss: 0.8788 - val_accuracy: 0.6149
Epoch 60/100
40/40 [==============================] - 7s 169ms/step - loss: 0.6243 - accuracy: 0.6684 - val_loss: 0.8793 - val_accuracy: 0.6117
Epoch 61/100
40/40 [==============================] - 7s 170ms/step - loss: 0.6277 - accuracy: 0.6641 - val_loss: 0.9263 - val_accuracy: 0.6038
Epoch 62/100
40/40 [==============================] - 7s 174ms/step - loss: 0.6198 - accuracy: 0.6766 - val_loss: 0.8902 - val_accuracy: 0.6117
Epoch 63/100
40/40 [==============================] - 7s 173ms/step - loss: 0.6249 - accuracy: 0.6691 - val_loss: 0.8062 - val_accuracy: 0.6212
Epoch 64/100
40/40 [==============================] - 7s 175ms/step - loss: 0.6206 - accuracy: 0.6672 - val_loss: 0.7508 - val_accuracy:

outcome: 64.98, 그러나 epoch 증가할때마다 성능 떨어짐

In [110]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'mel_shape': (100, 313, 1),
 'cqt_shape': (80, 157, 1),
 'stft_shape': (257, 313, 1),
 'model1': 'toy0_1_5',
 'model2': 'toy0_2_5',
 'model_fnm1': 'tmp_model0_5/toy0_1_5_model1.hdf5',
 'model_fnm2': 'tmp_model0_5/toy0_2_5_model2.hdf5'}

In [111]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape


In [112]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'toy0_1_6', m_name2 = 'toy0_2_6', param_feature = params_feature)

1

In [113]:
## 우리 모형 저장된 폴더이름
model_folder

'tmp_model0_6'

In [114]:
## 테스트 데이터에 모형 돌려서 스코어 저장할 폴더
output_folder = '/Data2/hmd/hmd_sy/notebooks/out_model0_6'

In [115]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 1)

Loading Challenge model...
Running model on Challenge data...
Done.


### 대회 평가 지표

In [116]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.772,0.590,0.503,0.791,0.687,18182.279

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.667,0.657,0.556,0.597,0.388,19853.373

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.863,0.626,0.827
AUPRC,0.720,0.147,0.903
F-measure,0.634,0.000,0.874
Accuracy,0.684,0.000,0.899

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.667,0.667
AUPRC,0.724,0.590
F-measure,0.421,0.691
Accuracy,0.286,0.925



In [117]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.3371869878586387
0.6548306714591623
0.32939043020418846
0.6706095713643978


### threshold 바꿔가면서 결과 출력

In [118]:


for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")


threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.772,0.590,0.111,0.199,0.512,15140.890

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.667,0.657,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.863,0.626,0.827
AUPRC,0.720,0.147,0.903
F-measure,0.332,0.000,0.000
Accuracy,1.000,0.000,0.000

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.667,0.667
AUPRC,0.724,0.590
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.772,0.590,0.131,0.220,0.523,14705.680

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.667,0.657,0.344,0.503,0.816,14915.104

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.863,0.626,0.827
AUPRC,0.720,0.147,0.903
F-measure,0.338,0.000,0.056
Accuracy,1.000,0.000,0.029

#Outcome scores (per class)
Classes,Abnormal,Normal
AU